##Homework 1:

Using chinook.db write pandas code.

In [38]:
import sqlite3 
import pandas as pd



In [39]:
# 1. Customer Purchases Analysis:
# Find the total amount spent by each customer on purchases (considering invoices).
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
customers = pd.read_sql_query("SELECT * FROM Customer", conn)
invoices = pd.read_sql_query("SELECT * FROM Invoice", conn)

total_spend = invoices.groupby("CustomerId")['Total'].sum().reset_index()
result = pd.merge(total_spend, customers, on="CustomerId")
result
# Identify the top 5 customers with the highest total purchase amounts.
top_five_highest_total_amounts = result.sort_values(by= 'Total', ascending=False).head(5)
top_five_highest_total_amounts

# Display the customer ID, name, and the total amount spent for the top 5 customers.
result['FullName'] = result['FirstName'] + " " + result['LastName']
result = result[['CustomerId', 'FullName', 'Total']].sort_values(by='Total', ascending=False).head(5)
result


,CustomerId,FullName,Total
5,6,Helena Holý,49.62
25,26,Richard Cunningham,47.62
56,57,Luis Rojas,46.62
44,45,Ladislav Kovács,45.62
45,46,Hugh O'Reilly,45.62


In [43]:
# 2. Album vs. Individual Track Purchases:
album = pd.read_sql_query("SELECT * FROM Album", conn)
album.head()
invoices_line = pd.read_sql_query("SELECT *FROM InvoiceLine", conn)
invoices_line.head()
tracks = pd.read_sql_query("SELECT *FROM Track", conn)
tracks.head()
merge = pd.merge(invoices_line, tracks, on="TrackId")
merge

album_track_counts = tracks.groupby("AlbumId")['TrackId'].count().reset_index()
album_track_counts.columns = ["AlbumId", "TotalTracksInAlbum"]
album_track_counts

invoice_album_group = merge.groupby(['InvoiceId',"AlbumId"])['TrackId'].count().reset_index()
invoice_album_group.columns = ['InvoiceId',"AlbumId", "TracksBought"]

joined = pd.merge(invoice_album_group, album_track_counts, on="AlbumId")
joined['IsFullAlbum'] = joined['TracksBought'] == joined['TotalTracksInAlbum']

invoices = pd.read_sql_query("SELECT InvoiceId, CustomerId FROM Invoice", conn)
joined = pd.merge(joined, invoices, on="InvoiceId") 

# Determine the percentage of customers who prefer to buy individual tracks instead of full albums.
customer_album_pref = joined.groupby("CustomerId")['IsFullAlbum'].any().reset_index()
customer_album_pref

individual_buyers = customer_album_pref[customer_album_pref['IsFullAlbum'] == False]
percentage_individual = (len(individual_buyers) / len(customer_album_pref)) * 100
print(f"📊 Percentage of customers who prefer individual tracks: {percentage_individual:.2f}%")

# A customer is considered to prefer individual tracks if they have purchased only a subset of tracks from an album.
total_customers = customer_album_pref.shape[0]
num_individual_only = individual_buyers.shape[0]
percentage = (num_individual_only / total_customers)*100
print(f"📊 Percentage of customers who prefer individual tracks (never vought a full album): {percentage:.2f}%")
# Provide a summary of the percentage of customers who fall into each category (individual tracks vs. full albums).
percentage_full_album = 100 - percentage
print(f"\nSummary: ")
print(f"📊 Full album buyers (at least once): {percentage_full_album: .2f}%")


📊 Percentage of customers who prefer individual tracks: 64.41%
📊 Percentage of customers who prefer individual tracks (never vought a full album): 64.41%

Summary: 
📊 Full album buyers (at least once):  35.59%
